In [139]:
import os
from dataclasses import dataclass
from pathlib import Path

In [140]:
%pwd
os.chdir("../end-to-end-chest-cancer-problem")
print(os.getcwd())

d:\end-to-end-chest-cancer-problem


In [141]:
@dataclass(frozen=True)
class PrepareBaseModelConfig:
    root_dir: Path
    base_model_path: Path
    updated_base_model_path: Path
    param_image_size: list
    param_learning_rate: float
    param_include_top: bool
    param_weights: str
    param_classes: int

In [ ]:
from cnnChestCancer.constants import *
from cnnChestCancer.utils.common import read_yaml, create_directories


class ConfigurationManager:
    def __init__(self, config_filepath = CONFIG_FILE_PATH , params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

        create_directories([self.config.artifacts_root])

    def get_prepare_base_model_config(self) -> PrepareBaseModelConfig:
        config = self.config.prepare_base_model

        create_directories([config.root_dir])

        prepare_base_model_config = PrepareBaseModelConfig(
            root_dir=Path(config.root_dir),
            base_model_path=Path(config.base_model_path),
            updated_base_model_path=Path(config.updated_base_model_path),
            param_image_size = self.params.IMAGE_SIZE,
            param_learning_rate= self.params.LEARNING_RATE,
            param_include_top= self.params.INCLUDE_TOP,
            param_weights= self.params.WEIGHTS,
            param_classes= self.params.CLASSES,
        )

        return prepare_base_model_config


In [143]:
import os
import urllib.request as request
from zipfile import ZipFile
import tensorflow as tf

In [ ]:
class PrepareBaseModel:
    def __init__(self,config : PrepareBaseModelConfig):
        self.config = config

    def get_base_model(self):
        self.model = tf.keras.applications.resnet50.ResNet50(
            input_shape=self.config.param_image_size,
            weights=self.config.param_weights,
            include_top=self.config.param_include_top
        )

        self.save_model(path=self.config.base_model_path, model=self.model)   

    @staticmethod
    def _prepare_full_model(b_model, classes, freeze_all, freeze_till, learning_rate):
        if freeze_all:
            for layer in b_model.layers:
                layer.trainable = False
        elif freeze_till is not None and freeze_till > 0:
            for layer in b_model.layers[:-freeze_till]:
                layer.trainable = False
            for layer in b_model.layers[-freeze_till:]:
                layer.trainable = True
        else:
        # default: all layers trainable
            for layer in b_model.layers:
                layer.trainable = True    


        x = tf.keras.layers.GlobalAveragePooling2D()(b_model.output)
        prediction = tf.keras.layers.Dense(
        units=classes,
        activation="softmax"
        )(x)

        
        full_model = tf.keras.models.Model(
            inputs=b_model.input,
            outputs=prediction
        )

        full_model.compile(
            optimizer=tf.keras.optimizers.Adam(learning_rate=learning_rate),
            loss=tf.keras.losses.CategoricalCrossentropy(),
            metrics=["accuracy"]
        )

        full_model.summary()  


        return full_model
    

    def update_base_model(self):
        self.full_model = self._prepare_full_model(
            b_model=self.model,
            classes=self.config.param_classes,
            freeze_all=False,
            freeze_till=32,
            learning_rate=self.config.param_learning_rate
        )

        self.save_model(path=self.config.updated_base_model_path, model=self.full_model)
        
    


    @staticmethod
    def save_model(path: Path, model: tf.keras.Model):
        model.save(path)         

[2025-11-22 01:50:23,302: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-11-22 01:50:23,311: INFO: common: yaml file: params.yaml loaded successfully]
[2025-11-22 01:50:23,311: INFO: common: created directory at: artifacts]
[2025-11-22 01:50:23,319: INFO: common: created directory at: artifacts/prepare_base_model]
[2025-11-22 01:50:27,940: WARNING: saving_utils: Compiled the loaded model, but the compiled metrics have yet to be built. `model.compile_metrics` will be empty until you train or evaluate the model.]
Model: "model_4"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_6 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                              